<a href="https://colab.research.google.com/github/raj-vijay/da/blob/master/11_Linear_Programming_Diet_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BACKGROUND**

<p align = 'justify'>This exercise you will implement two models that can be solved using linear programming. First, you will look at the diet problem and compare the linear programming solution to the integer linear programming solution. In task 3 you will also implement the transport problem as linear programming model.</p>

**Task 1 (diet problem)**

<p align = 'justify'>Let’s assume a farmer can feed the animals using two different products, which have different nutritional composition and come at different costs as follows:</p>


| Feed | Energy | Protein | Calcium | Cost |
| ---- | ----   | ------- | ------- | ---- |
| A    | 2      | 5       | 4       | 9    |
| B    | 4      | 3       | 1       | 7    |

<p align = 'justify'>In order to achieve the required quality of meat the farmer needs to ensure that the following minimum nutritional requirements are met:</p>


| Energy | Protein | Calcium |
| ------ | ------- | ------- |
| 12     | 15      | 8       |

<p align = 'justify'>Define a Linear Program that optimises the cost of the mixture of feeds while at the same time meeting the minimum nutritional requirements.</p>

In [ ]:
!pip install ortools

     |████████████████████████████████| 14.0MB 305kB/s 
     |████████████████████████████████| 1.0MB 25.2MB/s 
     |████████████████████████████████| 133kB 56.7MB/s 
ERROR: tensorflow-metadata 0.25.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0


In [ ]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [ ]:
def diet():
    nutrition_contents = pd.DataFrame([[2,5,4,9],[4,3,1,7]], index=["Feed A","Feed B"], columns=["Energy","Protein","Calcium","Cost"])    
    print(nutrition_contents)
    print()
    
    nutrition_requirements = pd.Series([12,15,8], index=["Energy","Protein","Calcium"])    
    print(nutrition_requirements)
    print()

    feeds = set(nutrition_contents.index)
    print(feeds)

    requirements = set(nutrition_requirements.index)
    print(requirements)
    print()
    
    feeding_amount = {}
    
    MIP = False
    if MIP:
        solver = pywraplp.Solver('LPWrapper',
                                 pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
        for feed in feeds:
            feeding_amount[feed] = solver.IntVar(0, solver.infinity(), feed)
    else:
        solver = pywraplp.Solver('LPWrapper', 
                             pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
        for feed in feeds:
            feeding_amount[feed] = solver.NumVar(0, solver.infinity(), feed)

        
    for requirement in requirements:
        c = solver.Constraint(float(nutrition_requirements[requirement]), solver.infinity())
        for feed in feeds:
            c.SetCoefficient(feeding_amount[feed], float(nutrition_contents[requirement][feed]))

    cost = solver.Objective()    
    for feed in feeds:
        cost.SetCoefficient(feeding_amount[feed], float(nutrition_contents["Cost"][feed]))
    cost.SetMinimization()
    solver.Solve()
    
    total_cost = 0
    for feed in feeds:
        print(feed, " -> ", feeding_amount[feed].solution_value())
        total_cost += feeding_amount[feed].solution_value()*nutrition_contents["Cost"][feed]
    print()                
    print("Total cost:", total_cost)

In [ ]:
diet()

        Energy  Protein  Calcium  Cost
Feed A       2        5        4     9
Feed B       4        3        1     7

Energy     12
Protein    15
Calcium     8
dtype: int64

{'Feed B', 'Feed A'}
{'Protein', 'Calcium', 'Energy'}

Feed B  ->  2.142857142857143
Feed A  ->  1.714285714285714

Total cost: 30.428571428571423


**Task 2 (mixed inter programming)**

<p align = 'justify'>Let’s now assume that the above problem is a once-off decision only, so it is not about determining the optimal mixture of feed products but about determining how many units of each to buy to satisfy demand. The problem then is an Integer Linear Program, with the decision variables being integers.

Use a Mixed Integer Programming solver to solve the above problem and observe the difference.</p>

**Task 3 (transport problem)**
<p align = 'justify'>Let’s assume there are two energy suppliers connected to the grid delivering the following amount of energy.

| Supplier   | Supply |
| ---------- | ------ |
| Supplier A | 6      |
| Supplier B | 9      |

This energy is to meet the consumer demand as follows

| Consumer   | Demand |
| ---------- | ------ |
| Consumer A | 8      |
| Consumer B | 5      |
| Consumer C | 2      |

Let’s further assume the DSO of the electricity grid charges the following transmission fees

| Supplier   | Consumer A | Consumer B | Consumer C |
| ---------- | ---------- | ---------- | ---------- |
| Supplier A | 5          | 5          | 3          |
| Supplier B | 6          | 4          | 1          |


Define a linear program that optimises the transmission costs between energy suppliers and consumers and determine the optimal energy mix for each consumer.

In [ ]:
def transport():
    transmission_costs = pd.DataFrame([[5,5,3],[6,4,1]],index=["Supplier A","Supplier B"], columns=["Consumer A","Consumer B", "Consumer C"])        
    print(transmission_costs)
    print()

    supply = pd.Series([6,9],index=["Supplier A","Supplier B"])
    print(supply)
    print()

    demand = pd.Series([8,5,2],index=["Consumer A","Consumer B", "Consumer C"])
    print(demand)
    print()

        
    suppliers = set(transmission_costs.index)
    consumers = set(transmission_costs.columns)
    
    print(suppliers)
    print(consumers)
    print()
    
    solver = pywraplp.Solver('LPWrapper', 
                             pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

    delivery = {}    
    for supplier in suppliers:
        for consumer in consumers:
            delivery[(supplier,consumer)] = solver.NumVar(0,solver.infinity(),supplier+"_" +consumer)


    # each supplier needs to supply all their energy
    for supplier in suppliers:
        c = solver.Constraint(float(supply[supplier]), solver.infinity())
        for consumer in consumers:
            c.SetCoefficient(delivery[(supplier,consumer)], 1)
            

    # each each consumer needs to have their demand met
    for consumer in consumers:
        c = solver.Constraint(float(demand[consumer]), solver.infinity())
        for supplier in suppliers:
            c.SetCoefficient(delivery[(supplier,consumer)], 1)

    cost = solver.Objective()
    for supplier in suppliers:
        for consumer in consumers:
            cost.SetCoefficient(delivery[(supplier,consumer)], float(transmission_costs[consumer][supplier]))
    cost.SetMinimization()
    solver.Solve()

    total_cost = 0
    for supplier in suppliers:
        for consumer in consumers:
            if delivery[(supplier,consumer)].solution_value()>0:
                print("Delivery from",supplier,"to",consumer,"is",delivery[(supplier,consumer)].solution_value())
                total_cost += delivery[(supplier,consumer)].solution_value()*transmission_costs[consumer][supplier]
    print()
    print("Total cost:",total_cost)

In [ ]:
transport()

            Consumer A  Consumer B  Consumer C
Supplier A           5           5           3
Supplier B           6           4           1

Supplier A    6
Supplier B    9
dtype: int64

Consumer A    8
Consumer B    5
Consumer C    2
dtype: int64

{'Supplier A', 'Supplier B'}
{'Consumer B', 'Consumer C', 'Consumer A'}

Delivery from Supplier A to Consumer A is 8.0
Delivery from Supplier B to Consumer B is 5.0
Delivery from Supplier B to Consumer C is 4.0

Total cost: 64.0
